In [1]:
import os
import numpy as np
import pandas as pd

In [4]:
new_df_1 = pd.read_csv(r'nonemerg_amb.csv')
new_df_2 = pd.read_csv(r'emerg_amb.csv')
new_df_3 = pd.read_csv(r'all_amb.csv')

In [6]:
new_df_1.columns=['zip','year','state_id','state_fips','tot_benes_nonemerg',
                 'tot_srvcs_nonemerg','tot_bene_day_srvcs_nonemerg','tot_srvcs_chrg_nonemerg']
new_df_2.columns=['zip','year','state_id','state_fips','tot_benes_emerg',
                 'tot_srvcs_emerg','tot_bene_day_srvcs_emerg','tot_srvcs_chrg_emerg']
new_df_3.columns=['zip','year','state_id','state_fips','tot_benes_allamb',
                 'tot_srvcs_allamb','tot_bene_day_srvcs_allamb','tot_srvcs_chrg_allamb']

In [7]:
len(new_df_1), len(new_df_2), len(new_df_3)

(19770, 53501, 55335)

In [8]:
temp_df = pd.merge(new_df_1, new_df_2, on=['zip','year','state_id','state_fips'], how='outer')
temp_df = pd.merge(temp_df, new_df_3, on=['zip','year','state_id','state_fips'], how='outer')
temp_df = temp_df.fillna(0)
temp_df.info()
amb_df = temp_df.copy()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55335 entries, 0 to 55334
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   zip                          55335 non-null  int64  
 1   year                         55335 non-null  int64  
 2   state_id                     55335 non-null  object 
 3   state_fips                   55335 non-null  int64  
 4   tot_benes_nonemerg           55335 non-null  float64
 5   tot_srvcs_nonemerg           55335 non-null  float64
 6   tot_bene_day_srvcs_nonemerg  55335 non-null  float64
 7   tot_srvcs_chrg_nonemerg      55335 non-null  float64
 8   tot_benes_emerg              55335 non-null  float64
 9   tot_srvcs_emerg              55335 non-null  float64
 10  tot_bene_day_srvcs_emerg     55335 non-null  float64
 11  tot_srvcs_chrg_emerg         55335 non-null  float64
 12  tot_benes_allamb             55335 non-null  int64  
 13  tot_srvcs_allamb

In [9]:
tele_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/telehealth_numphysicians_numservices.csv')

In [11]:
temp_df = pd.merge(amb_df, tele_df, on=['zip','year','state_id','state_fips'], how='outer')
temp_df = temp_df.fillna(0)
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58177 entries, 0 to 58176
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   zip                          58177 non-null  int64  
 1   year                         58177 non-null  int64  
 2   state_id                     58177 non-null  object 
 3   state_fips                   58177 non-null  int64  
 4   tot_benes_nonemerg           58177 non-null  float64
 5   tot_srvcs_nonemerg           58177 non-null  float64
 6   tot_bene_day_srvcs_nonemerg  58177 non-null  float64
 7   tot_srvcs_chrg_nonemerg      58177 non-null  float64
 8   tot_benes_emerg              58177 non-null  float64
 9   tot_srvcs_emerg              58177 non-null  float64
 10  tot_bene_day_srvcs_emerg     58177 non-null  float64
 11  tot_srvcs_chrg_emerg         58177 non-null  float64
 12  tot_benes_allamb             58177 non-null  float64
 13  tot_srvcs_allamb

##### From zip to county

In [15]:
amb_tele_df = pd.read_csv(r'amb_tele_provider_service.csv')

In [12]:
# group doctors by county
zip_county_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/ziptocounty/ZIP_COUNTY.csv')
zip_county_df.columns=['zip','county','county_tot_ratio','year']

In [13]:
# keep only the counties that have max county_tot_ratio
zip_df_gb = zip_county_df.groupby(['zip','year']).max().reset_index(level=[0,1])[['zip','year','county_tot_ratio']].drop_duplicates()
zip_county_df = pd.merge(zip_county_df, zip_df_gb, on=['zip','year','county_tot_ratio'])
# If it's 50:50, keep the most frequent value by zip code
zip_county_df = zip_county_df.groupby(['zip']).agg(pd.Series.mode).reset_index(level=[0])
zip_county_df = zip_county_df[['zip','county']]

In [16]:
len(amb_tele_df), len(zip_county_df)

(58177, 39644)

In [17]:
temp_df = pd.merge(amb_tele_df, zip_county_df, on=['zip'], how='outer')
temp_df = temp_df[~temp_df['year'].isna()].reset_index(drop=True)
temp_df = temp_df[~temp_df['county'].isna()].reset_index(drop=True) # check
temp_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58175 entries, 0 to 58174
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   zip                          58175 non-null  int64  
 1   year                         58175 non-null  float64
 2   state_id                     58175 non-null  object 
 3   state_fips                   58175 non-null  float64
 4   tot_benes_nonemerg           58175 non-null  float64
 5   tot_srvcs_nonemerg           58175 non-null  float64
 6   tot_bene_day_srvcs_nonemerg  58175 non-null  float64
 7   tot_srvcs_chrg_nonemerg      58175 non-null  float64
 8   tot_benes_emerg              58175 non-null  float64
 9   tot_srvcs_emerg              58175 non-null  float64
 10  tot_bene_day_srvcs_emerg     58175 non-null  float64
 11  tot_srvcs_chrg_emerg         58175 non-null  float64
 12  tot_benes_allamb             58175 non-null  float64
 13  tot_srvcs_allamb

/var/folders/ff/7xvchf4558v3w_jjs11bdwlw0000gp/T/ipykernel_97062/1718783504.py:4: FutureWarning: null_counts is deprecated. Use show_counts instead
  temp_df.info(null_counts=True)


In [18]:
temp_df = temp_df.rename(columns={'county':'fips'})

In [19]:
temp_df_gb = temp_df.drop(columns=['zip']).groupby(['fips','state_id','state_fips','year']).sum().reset_index(level=[0,1,2,3])

In [27]:

# Counties that exist at least once (in the full data set w/ all counties in the US)
all_c = sorted(temp_df_gb.fips.value_counts().index.tolist())
# All years
all_y = [2013,2014,2015,2016,2017,2018,2019]
# Together
df = pd.concat([pd.DataFrame(data=np.repeat(np.array(all_c), repeats=len(all_y)), columns=['fips']),
                pd.DataFrame(data=np.tile(np.array(all_y), reps=len(all_c)), columns=['year'])], axis=1)


In [28]:
df

,fips,year
0,1001,2013
1,1001,2014
2,1001,2015
3,1001,2016
4,1001,2017
...,...,...
19392,56045,2015
19393,56045,2016
19394,56045,2017
19395,56045,2018


In [29]:

# Whether a county implemented telehealth in that year
new_list = []
for i in range(len(df)):
    try:
        if temp_df[(temp_df['year']==df['year'][i])&(temp_df['fips']==df['fips'][i])]['tot_benes_alltele'].values[0]==0:
            new_list.append(0)
        elif temp_df[(temp_df['year']==df['year'][i])&(temp_df['fips']==df['fips'][i])]['tot_benes_alltele'].values[0]!=0:
            new_list.append(1)
    except Exception as e:
        new_list.append(0)
df['tele']=new_list


In [30]:
# These counties did not adopt telehealth; they can be used as the control group
cg_df = df.groupby(['fips']).sum().reset_index(level=[0])
cg_list = cg_df[cg_df['tele']==0].fips.values.tolist()


In [32]:
len(cg_list)

2248

In [33]:
# These counties adopted telehealth 
tg_df = df[(~df['fips'].isin(cg_list))&(df['tele']==1)].groupby(['fips']).min().reset_index(level=[0])
# Let's see when they each begin to implement telehealth
tg_dict = {}
for i in range(len(tg_df)):
    tg_dict[tg_df['fips'][i]] = tg_df['year'][i]


In [34]:
tg_df.groupby(['year']).size()
# We may better use 2015; when there were 60 new adopters
# Or 2016; when there weree 69 new adopters


year
2013    107
2014     48
2015     62
2016     70
2017     76
2018     81
2019     79
dtype: int64

----

In [36]:
tg_list = tg_df[tg_df['year']>=2015].fips.values.tolist()

In [39]:
len(tg_list), len(cg_list) # 368, 2248

(368, 2248)

In [40]:
temp_df_gb = temp_df.drop(columns=['zip']).groupby(['fips','state_id','state_fips','year']).sum().reset_index(level=[0,1,2,3])

In [41]:
temp_df_gb

,fips,state_id,state_fips,year,tot_benes_nonemerg,tot_srvcs_nonemerg,tot_bene_day_srvcs_nonemerg,tot_srvcs_chrg_nonemerg,tot_benes_emerg,tot_srvcs_emerg,...,numphysicians_tele_q3014,numphysicians_tele_all,tot_benes_q3014,tot_srvcs_q3014,tot_bene_day_srvcs_q3014,tot_srvcs_chrg_q3014,tot_benes_alltele,tot_srvcs_alltele,tot_bene_day_srvcs_alltele,tot_srvcs_chrg_alltele
0,1001,AL,1.0,2013.0,0.0,0.0,0.0,0.0,469.0,538.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,AL,1.0,2014.0,0.0,0.0,0.0,0.0,457.0,532.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001,AL,1.0,2015.0,0.0,0.0,0.0,0.0,520.0,648.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001,AL,1.0,2016.0,0.0,0.0,0.0,0.0,496.0,626.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001,AL,1.0,2017.0,0.0,0.0,0.0,0.0,540.0,704.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18469,56045,WY,56.0,2015.0,18.0,18.0,18.0,9000.0,107.0,141.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18470,56045,WY,56.0,2016.0,24.0,25.0,25.0,15275.0,137.0,214.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18471,56045,WY,56.0,2017.0,36.0,38.0,37.0,25125.0,127.0,176.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18472,56045,WY,56.0,2018.0,11.0,11.0,11.0,6875.0,116.0,167.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df_0 = df[df['fips'].isin(tg_list+cg_list)].reset_index(drop=True)
df_0['tg'] = [1 if i in tg_list else 0 for i in df_0['fips']]
new_df = pd.merge(df_0, temp_df_gb[['fips','year','tot_benes_nonemerg',
                                  'tot_srvcs_nonemerg', 'tot_bene_day_srvcs_nonemerg',
                                  'tot_srvcs_chrg_nonemerg', 'tot_benes_emerg', 'tot_srvcs_emerg',
                                  'tot_bene_day_srvcs_emerg', 'tot_srvcs_chrg_emerg', 'tot_benes_allamb',
                                  'tot_srvcs_allamb', 'tot_bene_day_srvcs_allamb',
                                  'tot_srvcs_chrg_allamb', 'tot_benes_q3014', 'tot_srvcs_q3014',
                                  'tot_bene_day_srvcs_q3014', 'tot_srvcs_chrg_q3014', 'tot_benes_alltele',
                                  'tot_srvcs_alltele', 'tot_bene_day_srvcs_alltele',
                                  'tot_srvcs_chrg_alltele']], on=['fips','year'], how='outer')
new_df = new_df[~new_df['tele'].isna()].reset_index(drop=True)
new_df = new_df.fillna(0)

In [43]:
df_0 = new_df.copy()
df_0.to_csv(r'did_tele_ambulatory_nomatching_county_2015.csv', index=False)